<h2 id="tocheading">Table of Contents</h2>
<div id="toc"></div>

In [6]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

### WRANGLE AND ANALYZE DATA PROJECT

#### Introduction

The aim of this project is to wrangle WeRateDogs Twitter data in order to create interesting and trustworthy analyses and visualisations.  
The data wrangling process consist of three phases 
    - Gathering data
    - Assessing data
    - Cleaning data
- The cleaned data will be analysed and used to create visuals to give possible interpretation for the data. A report for both processes will also be submitted at the end. 

#### Gathering Data
Three pieces of data is required for this project:

1. The WeRateDogs Twitter archive data, provided by Udacity and downloaded manually from Udacity resource center
2. The tweet image predictions, hosted on Udacity's server and downloaded programmatically using `Get Requests` 
3. Retweet and favorite counts for the tweet_id's in the archived data from `1` above. This data is accessed by qerying the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file.

#### Assessing Data 
- After gathering each of the above pieces of data, assess them visually and programmatically for quality and tidiness issues.
- Target is to  detect and document at least eight (8) quality issues and two (2) tidiness issues.

#### Cleaning Data
- Issues documented while assessing wil be cleaned to give an output of high quality and tidy master pandas DataFrame.

#### Storing, Analyzing, and Visualizing Data 
- Store the clean DataFrame(s) in a CSV file with the main one named `twitter_archive_master.csv`. Wrangled data will be analysed and visualised using Jupyter notebook. This should include at least three (3) insights and one (1) visualisation. 
- A written report will prepared to describe the wrangling efforts and second report to communicate the insights and displays will also be prepared. 

In [126]:
#Import Necessary modules
import pandas as pd
import numpy as np
import os
#to make requests
import requests
#to display tables
from IPython.display import display
#to access twitter APi
import tweepy as tw

#to write json to pandas dataframe
from pandas import DataFrame

#for json file
import json


In [127]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style> 

### DATA GATHERING

Three pieces of data will be gathered for this project.

**Data One;** The WeRateDogs Twitter archive. Provided and made available by Udacity. Downloaded from the resource centre and loaded into notebook as **.csv**

In [177]:
#import data to have an overview
df1 = pd.read_csv('twitter_archive_enhanced.csv')

In [178]:
df1.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [117]:
df1.shape

(2356, 17)

**Data Two;** The tweet image predictions, this information tells what breed of dog (or other object, animal, etc.) is present in each tweet. This was already provided by Udacity according to a neural network that can classify breeds of dogs. 

- This file (image_predictions.tsv) is hosted on Udacity's servers and downloaded using **Requests** library

In [6]:
#Getting the url
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

In [7]:
#to view raw data
#response.content

In [53]:
#savingg file to computer
with open('C:/Users/Frances-Anthony/Documents/Udacity/data_wrangle_analyze_project/image_predictions.tsv', mode='wb') as file:
    file.write(response.content)

In [12]:
#Load image data
df2 = pd.read_csv('image_predictions.tsv', delimiter="\t")
df2.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


**Data three;** Extract each tweet's retweet count and favorite ("like") count for the `tweet_id` in the archived data downloaded. 

**HOW:** 
1. using the tweet IDs in the `WeRateDogs` Twitter archive, 
2. query the Twitter API for each tweet's JSON data using Python's `Tweepy library` and 
3. store each tweet's entire set of JSON data in a file called `tweet_json.txt file.`
4. Read the `.txt` file line by line into a pandas frame with these columns (tweet_id, retweet_count, favorite_count).

In [ ]:
#import necessary modules in this insatnce Tweepy has already been imported above
#from the twitter archved data provided by Udacity call tweet_id to query twitter API for retweets and "likes" counts
#to access twitter API, input consumer and secret key gotten from twitter
#for privacy, i will leave them blank

#define keys
consumer_key= '.......'
consumer_secret = '..........'
access_token = '..........'
access_token_secret = '........'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

#get tweets from twitter API 
retweet_favorite_count = []

#save missing tweets in this list
not_found = []

with open('tweet_json.txt', mode = "w") as file:
    for i in list(df1.tweet_id):
        try:
            tweet = api.get_status(str(i))
            file.write(json.dumps(tweet._json))
            retweet_favorite_count.append({"tweet_id":str(i),
                                          "retweet_count": tweet._json['retweet_count'],
                                          "favourite_count": tweet._json['favorite_count']})
        except:
                not_found.append(i)

In [90]:
#Read the .txt file line by line into a pandas frame with these columns 
#(tweet_id, retweet_count, favorite count).
#df3 = pd.DataFrame(retweet_favorite_count, columns=['tweet_id', 'retweet_count', 'favourite_count'])

In [66]:
#write to a csv file
#df3.to_csv('retweet_and_favorites_counts.csv')

In [13]:
#read retweet_favorite_counts as csv and assign to dataframe
df3 = pd.read_csv('retweet_and_favorite_counts.csv')
df3.head()

,Unnamed: 0,tweet_id,retweet_count,favourite_count
0,0,892420643555336193,7340,34978
1,1,892177421306343426,5475,30279
2,2,891815181378084864,3621,22784
3,3,891689557279858688,7529,38245
4,4,891327558926688256,8108,36522


In [14]:
df3.shape

(2331, 4)

## ASSESSING


The next phase of the project is assessing the gathered data. I will be assessing the data for quality and tidiness both manually and programmatically using pandas methods.


#### 1. ASSESSING TWITTER ARCHIVE DATA

In [179]:
#renamed the df 
archived_data = df1
archived_data.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


#### COLUMN DESCRIPTION
- tweet_id: 
- n_reply_to_status_id	
- in_reply_to_user_id	
- timestamp	
- source	
- text	
- retweeted_status_id	
- retweeted_status_user_id	
- retweeted_status_timestamp	
- expanded_urls	
- rating_numerator	
- rating_denominator	
- name	
- doggo	
- floofer
- pupper
- puppo

### QUALITY ISSUES
- missing data: n_reply_to_status_id and in_reply_to_user_id has only 78 rows available out of the 2356. Possibly drop these columns
- not original tweet, columns retweeted_status_timestamp, retweeted status_id and user_id means that rows are retweet and not original tweet. (drop rows)	
- dog name is none for 745 rows - check if the same rows with retweet
- dog names recorded as `a` or `an` should be `None`
- rows with `a` also have dog style
- does not contain retweet and favorite counts 
- inconsistent rating_denominator -value should be 10, values    greater than 10 should be removed
- very high rating_numerator as much 1776, not an issue but keep in mind
- time stamp has object data type, change to datetime
- 

### TIDINESS ISSUES
- create one column for dog stage, collapse multiple colunmns and rows. convert the rows (name, doggo, floofer,pupper,puppo into two columns, one with dog name and one with dog level specifying either from the list)
- expanded url has multiple url on one row
- the three datasets can be one,all have tweet_id

In [16]:
archived_data.shape

(2356, 17)

In [17]:
archived_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [332]:
archived_data.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [18]:
archived_data.puppo.unique()

array(['None', 'puppo'], dtype=object)

In [19]:
archived_data.puppo.value_counts()

None     2326
puppo      30
Name: puppo, dtype: int64

In [20]:
archived_data.retweeted_status_id.nunique() 

181

In [21]:
#archived_data.rating_denominator.isnull()
archived_data.rating_numerator.isnull().sum().any()

False

In [22]:
archived_data.name.nunique(),archived_data.name.value_counts();

In [23]:
#check for duplicates
archived_data.tweet_id.duplicated().sum()

0

In [24]:
archived_data.tweet_id.nunique() #no duplicates

2356

In [25]:
#archived_data.name.isnull().any()
archived_data.query('name == "None"');

In [26]:
#dog names recorded as a or an
#archived_data.query('name == "an"')
archived_data.query('name == "a"');

In [27]:
archived_data.retweeted_status_id.notnull().sum()

181

In [29]:
#confirm if rows possible to be retweets contains dog names
#rchived_data[archived_data['retweeted_status_id'].notnull()]

In [348]:
archived_data.rating_denominator.nunique(),archived_data.rating_denominator.duplicated().sum()

(18, 2338)

In [349]:
#archived_data.rating_denominator.value_counts()
archived_data.rating_denominator.isnull().sum().any()

False

In [350]:
archived_data.rating_denominator.unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2], dtype=int64)

In [32]:
archived_data.rating_denominator.value_counts();

In [352]:
#archived_data.rating_numerator.notnull().sum()
archived_data.rating_numerator.isnull().sum()

0

In [33]:
archived_data.rating_numerator.unique();

In [35]:
archived_data.rating_numerator.value_counts();

In [36]:
archived_data.query('rating_numerator == 1776')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
979,749981277374128128,NaN,NaN,2016-07-04 15:00:45 +0000,"<a href=""https://about.twitter.com/products/tw...",This is Atticus. He's quite simply America af....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749981277...,1776,10,Atticus,None,None,None,None


In [37]:
archived_data.rating_numerator.isnull().sum()

0

In [38]:
archived_data.expanded_urls.isnull();

#### 2. ASSESSING IMAGE PREDICTIONS DATA

In [39]:
#Reassign dataframe
image_pred = df2
image_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [40]:
image_pred.shape

(2075, 12)

#### Column Description

- `tweet_id` is the last part of the tweet URL after "status/" 
- `p1` is the algorithm's #1 prediction for the image in the tweet
- `p1_conf` is how confident the algorithm is in its #1 prediction in `%`
- `p1_dog` is whether or not the #1 prediction is a breed of dog
- `p2` is the algorithm's second most likely prediction
- `p2_conf` is how confident the algorithm is in its #2 prediction 
- `p2_dog` is whether or not the #2 prediction is a breed of dog
- `p3` is the algorithm's 3rd most likely prediction
- `p3_conf` is how confident the algorithm is in its #3 prediction 
- `p3_dog` is whether or not the #3 prediction is a breed of dog

In [41]:
image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


#### Quality issues
- None

#### Tidiness issues
- None

In [50]:
sum(image_pred.tweet_id.duplicated())

0

#### 3. ASSESSING RETWEET AND FAVORITE COUNT DATA

In [43]:
rtwt_fav_count = df3
rtwt_fav_count.head()

,Unnamed: 0,tweet_id,retweet_count,favourite_count
0,0,892420643555336193,7340,34978
1,1,892177421306343426,5475,30279
2,2,891815181378084864,3621,22784
3,3,891689557279858688,7529,38245
4,4,891327558926688256,8108,36522


#### Column Description
- `tweet_id` the tweet_id
- `retweet_count` number of retweet for each Id
- `favourite_count` number of `likes` for the Id

In [44]:
rtwt_fav_count.shape

(2331, 4)

In [53]:
rtwt_fav_count.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   Unnamed: 0       2331 non-null   int64
 1   tweet_id         2331 non-null   int64
 2   retweet_count    2331 non-null   int64
 3   favourite_count  2331 non-null   int64
dtypes: int64(4)
memory usage: 73.0 KB


#### Quality issues
- first index column not needed - drop

#### Tidiness issues
- None

In [54]:
rtwt_fav_count.isnull().sum().any()

False

### CLEANING

In [191]:
df1 = pd.read_csv('twitter_archive_enhanced.csv')

In [192]:
archived_data = df1

In [193]:
#Make copies of dataframe
archived_data_clean = archived_data
image_pred_clean = image_pred
rtwt_fav_count_clean = rtwt_fav_count

In [185]:
archived_data_clean.head(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None


<font color='red'> **TIDINESS ISSUE - 1** </font>

**Define**

- Name and dog stages row do not obey [Tidy Data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html) rule.
- Use pandas method `assign`: To melt the dog stages columns that occured in multiple columns **doggo, floofer, pupper, puppo** to obey the tidy data rule. 

**Clean**

In [194]:
#create a copy for df
archived_data_clean_melt = archived_data_clean

In [169]:
#count for values in individual columns
archived_data_clean_melt['dog_stage'] = None
archived_data_clean_melt['dog_stage'] = archived_data_clean_melt.doggo + archived_data_clean_melt.floofer + archived_data_clean_melt.pupper + archived_data_clean_melt.puppo
archived_data_clean_melt['dog_stage'].value_counts()

NoneNoneNoneNone        1976
NoneNonepupperNone       245
doggoNoneNoneNone         83
NoneNoneNonepuppo         29
doggoNonepupperNone       12
NoneflooferNoneNone        9
doggoNoneNonepuppo         1
doggoflooferNoneNone       1
Name: dog_stage, dtype: int64

In [201]:
#use assign method to reassign the vlaues to the new column called dog_stage
archived_data_clean_melt = archived_data_clean_melt.assign(dog_stage = archived_data_clean_melt.doggo.astype(str) + archived_data_clean_melt.floofer.astype(str) + 
archived_data_clean_melt.pupper.astype(str) + archived_data_clean_melt.puppo.astype(str))                                             

In [202]:
archived_data_clean_melt.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,dog_stage
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,NoneNoneNoneNone
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,NoneNoneNoneNone
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,NoneNoneNoneNone
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,NoneNoneNoneNone
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,NoneNoneNoneNone


In [203]:
#replace all None with empty
archived_data_clean_melt['dog_stage'] = archived_data_clean_melt['dog_stage'].map(lambda x: x.replace("None",""))

In [204]:
archived_data_clean_melt.sample(3)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,dog_stage
522,809808892968534016,NaN,NaN,2016-12-16 17:14:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Maximus. His face is st...,7.939622e+17,4.196984e+09,2016-11-02 23:45:19 +0000,https://twitter.com/dog_rates/status/793962221...,12,10,Maximus,None,None,None,None,
655,791784077045166082,NaN,NaN,2016-10-27 23:30:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: I'm not sure what this dog is d...,6.820881e+17,4.196984e+09,2015-12-30 06:37:25 +0000,"https://vine.co/v/iqMjlxULzbn,https://vine.co/...",12,10,None,None,None,None,None,
939,753039830821511168,NaN,NaN,2016-07-13 01:34:21 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",So this just changed my life. 13/10 please enj...,NaN,NaN,NaN,https://vine.co/v/5W2Dg3XPX7a,13,10,None,None,None,None,None,


In [205]:
#drop other stage columns
archived_data_clean_melt.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis =1, inplace=True)
archived_data_clean_melt.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_stage
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,


In [206]:
#view unique values in column
archived_data_clean_melt.dog_stage.unique()

array(['', 'doggo', 'puppo', 'pupper', 'floofer', 'doggopuppo',
       'doggofloofer', 'doggopupper'], dtype=object)

In [207]:
#delete rows with empty dog stage
archived_data_clean_melt = archived_data_clean_melt[(archived_data_clean_melt.dog_stage != "")] 

In [212]:
archived_data_clean_melt.dog_stage.unique()

array(['doggo', 'puppo', 'pupper', 'floofer', 'doggopuppo',
       'doggofloofer', 'doggopupper'], dtype=object)

In [208]:
archived_data_clean_melt.shape

(380, 14)

**Test**

In [213]:
archived_data_clean_melt['dog_stage'].value_counts()

pupper          245
doggo            83
puppo            29
doggopupper      12
floofer           9
doggopuppo        1
doggofloofer      1
Name: dog_stage, dtype: int64

<font color='blue'> **QUALITY ISSUE - 1** </font>

**Define**

Some dog names recorded as `a` or `an` probably an error due to missign data. Rename as `None`

**Code**

In [216]:
archived_data_clean.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [217]:
archived_data_clean.query('name == "a"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
56,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a pupper approaching maximum borkdrive...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004...,14,10,a,None,None,pupper,None
649,792913359805018113,NaN,NaN,2016-10-31 02:17:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a perfect example of someone who has t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/792913359...,13,10,a,None,None,None,None
801,772581559778025472,NaN,NaN,2016-09-04 23:46:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Guys this is getting so out of hand. We only r...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/772581559...,10,10,a,None,None,None,None
1002,747885874273214464,NaN,NaN,2016-06-28 20:14:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a mighty rare blue-tailed hammer sherk...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747885874...,8,10,a,None,None,None,None
1004,747816857231626240,NaN,NaN,2016-06-28 15:40:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Viewer discretion is advised. This is a terrib...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/747816857...,4,10,a,None,None,None,None
1017,746872823977771008,NaN,NaN,2016-06-26 01:08:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a carrot. We only rate dogs. Please on...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/746872823...,11,10,a,None,None,None,None
1049,743222593470234624,NaN,NaN,2016-06-15 23:24:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a very rare Great Alaskan Bush Pupper....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/743222593...,12,10,a,None,None,pupper,None
1193,717537687239008257,NaN,NaN,2016-04-06 02:21:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",People please. This is a Deadly Mediterranean ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/717537687...,11,10,a,None,None,None,None
1207,715733265223708672,NaN,NaN,2016-04-01 02:51:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a taco. We only rate dogs. Please only...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/715733265...,10,10,a,None,None,None,None
1340,704859558691414016,NaN,NaN,2016-03-02 02:43:09 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a heartbreaking scene of an incredible...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/704859558...,10,10,a,None,None,pupper,None


In [218]:
archived_data_clean.query('name == "an"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
759,778396591732486144,NaN,NaN,2016-09-21 00:53:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is an East African Chalupa...,7.030419e+17,4.196984e+09,2016-02-26 02:20:37 +0000,https://twitter.com/dog_rates/status/703041949...,10,10,an,None,None,None,None
1025,746369468511756288,NaN,NaN,2016-06-24 15:48:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an Iraqi Speed Kangaroo. It is not a d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/746369468...,9,10,an,None,None,None,None
1362,703041949650034688,NaN,NaN,2016-02-26 02:20:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an East African Chalupa Seal. We only ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/703041949...,10,10,an,None,None,None,None
2204,668636665813057536,NaN,NaN,2015-11-23 03:46:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an Irish Rigatoni terrier named Berta....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/668636665...,10,10,an,None,None,None,None
2333,666337882303524864,NaN,NaN,2015-11-16 19:31:45 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an extremely rare horned Parthenon. No...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666337882...,9,10,an,None,None,None,None
2335,666287406224695296,NaN,NaN,2015-11-16 16:11:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an Albanian 3 1/2 legged Episcopalian...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666287406...,1,2,an,None,None,None,None
2349,666051853826850816,NaN,NaN,2015-11-16 00:35:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is an odd dog. Hard on the outside but lo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666051853...,2,10,an,None,None,None,None


In [219]:
#rename rows with name as 'a' and 'an' to None
archived_data_clean.loc[archived_data_clean['name'] == "a", 'name'] = 'None'
archived_data_clean.loc[archived_data_clean['name'] == "an", 'name'] = 'None'

**Test**

In [224]:
archived_data_clean.query('name == "an"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


In [223]:
archived_data_clean.query('name == "a"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


<font color='blue'> **QUALITY ISSUE - 2** </font>

**Define**

In-consistent `rating_denominator`. Denominator cannot be greater then **10**. Select rows and drop

**Code**

In [234]:
not_ten = list(archived_data_clean.query('rating_denominator !=10').index)
archived_data_clean.drop(index=not_ten, inplace = True)

In [235]:
archived_data_clean.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [236]:
archived_data_clean.rating_denominator.unique()

array([10], dtype=int64)

In [233]:
#archived_data_clean.drop(archived_data_clean.index[archived_data_clean['rating_denominator'] > 10])

**TEST**

In [237]:
archived_data_clean.rating_denominator.unique()

array([10], dtype=int64)

In [238]:
archived_data_clean.rating_denominator.value_counts()

10    2333
Name: rating_denominator, dtype: int64

<font color='blue'> **QUALITY ISSUE - 3** </font>